In [1]:
import numpy as np
import os
import networkx as nx
from scipy.ndimage import zoom
def load_and_validate_data(file_path):
    """
    Load data from a file and return it.
    Return None if data is missing.
    """
    try:
        loaded_data = np.load(file_path)
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def combine_data_from_folder(folder_path, batch_size=100):
    """
    Combine data from all valid files in the folder in batches.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        combined_ppg.append(ppg_f)
        combined_ecg.append(ecg_f)
        combined_seg_dbp.append(seg_dbp)
        combined_seg_sbp.append(seg_sbp)
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.concatenate(combined_ppg, axis=0)
            combined_ecg = np.concatenate(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.concatenate(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.concatenate(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

In [2]:
def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image.flatten()

def process_signal(i, ppg_signal, vg_image_size):
    """
    Generate a VG image for a given PPG signal.
    """
    print(f"VG img {i + 1}")
    ppg_signal = ppg_signal.flatten()
    vg_image = generate_vg_image(ppg_signal, vg_image_size)
    return vg_image

vg_image_size=224

In [3]:
import numpy as np
from scipy.signal import find_peaks

def select_n_peak_window(ppg_signal, n_peaks=3):
    ppg_3peak=[]
    for ppg in ppg_signal:
        x=ppg.flatten()
        peaks, _ = find_peaks(x, distance=50)  
    
    
    
        if len(peaks) < n_peaks:
            raise ValueError(f"Not enough peaks detected. Detected peaks: {len(peaks)}")
    
    
        start_index = peaks[0]
        end_index = peaks[n_peaks - 1]  
    
        ppg_window = x[start_index:end_index + 1]
        ppg_3peak.append(np.array(ppg_window))
    return ppg_3peak

def reshape_ppg_3_peaks(ppg_signal):
    ppg_3_peak_reshaped = []

    for ppg_window in ppg_signal:
        reshaped_window = ppg_window.reshape(1, -1)  # Reshape to (1, N)
        ppg_3_peak_reshaped.append(np.array(reshaped_window)) 
    return ppg_3_peak_reshaped

In [4]:
train_data_generator = combine_data_from_folder(train_dir, batch_size=10)

In [ ]:
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor


output_dir = 'Train_VG'
os.makedirs(output_dir, exist_ok=True)

for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(train_data_generator):
    
    output_file = os.path.join(output_dir, f'Train_VG_FP_batch_{batch_idx + 1}.npz')
    if os.path.exists(output_file):
        print(f"Batch {batch_idx + 1} already processed. Skipping...")
        continue
        
    print(f"Processing Batch {batch_idx + 1}...")
    print(len(combined_seg_dbp_batch))
    #ppg_peaks_3=select_n_peak_window(combined_ppg_batch)
    #ppg_peaks_3_reshaped=reshape_ppg_3_peaks(ppg_peaks_3)
    with ThreadPoolExecutor(max_workers=2) as executor:
        
        vg_images = list(executor.map(process_signal, range(len(combined_ppg_batch)), combined_ppg_batch, [vg_image_size]*len(combined_ppg_batch)))
        
    
    np.savez_compressed(output_file, vg_images=vg_images)
    print(f"Batch {batch_idx + 1} processing complete.")


Processing Batch 1...
5078
VG img 1
VG img 2
VG img 3
VG img 4
VG img 5
VG img 6
VG img 7
VG img 8
VG img 9
VG img 10
VG img 11


In [ ]:
val_data_generator = combine_data_from_folder(val_dir, batch_size=10)

In [ ]:
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor


output_dir = 'Val_VG'
os.makedirs(output_dir, exist_ok=True)


for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
    
    output_file = os.path.join(output_dir, f'Val_VG_batch_{batch_idx + 1}.npz')
    if os.path.exists(output_file):
        print(f"Batch {batch_idx + 1} already processed. Skipping...")
        continue
        
    print(f"Processing Batch {batch_idx + 1}...")
    print(len(combined_seg_dbp_batch))
    ppg_peaks_3=select_n_peak_window(combined_ppg_batch)
    ppg_peaks_3_reshaped=reshape_ppg_3_peaks(ppg_peaks_3)
    with ThreadPoolExecutor(max_workers=2) as executor:
        
        vg_images = list(executor.map(process_signal, range(len(ppg_peaks_3_reshaped)), ppg_peaks_3_reshaped, [vg_image_size]*len(ppg_peaks_3_reshaped)))
        
    
    np.savez_compressed(output_file, vg_images=vg_images)
    print(f"Batch {batch_idx + 1} processing complete.")


In [ ]:
test_data_generator = combine_data_from_folder(test_dir, batch_size=10)

In [ ]:
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor


output_dir = 'Test_VG'
os.makedirs(output_dir, exist_ok=True)


for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(test_data_generator):
    
    output_file = os.path.join(output_dir, f'Test_VG_batch_{batch_idx + 1}.npz')
    if os.path.exists(output_file):
        print(f"Batch {batch_idx + 1} already processed. Skipping...")
        continue
        
    print(f"Processing Batch {batch_idx + 1}...")
    print(len(combined_seg_dbp_batch))
    ppg_peaks_3=select_n_peak_window(combined_ppg_batch)
    ppg_peaks_3_reshaped=reshape_ppg_3_peaks(ppg_peaks_3)
    with ThreadPoolExecutor() as executor:
        
        vg_images = list(executor.map(process_signal, range(len(ppg_peaks_3_reshaped)), ppg_peaks_3_reshaped, [vg_image_size]*len(ppg_peaks_3_reshaped)))
        
    
    np.savez_compressed(output_file, vg_images=vg_images)
    print(f"Batch {batch_idx + 1} processing complete.")
